# In this notebook 

### We will employ textblob and vaddersentiment to annotate our textual data

In [4]:
import pandas as pd

df = pd.read_csv('../data/clean_data/clean_data.csv')
print(df.shape)
df.head(1)

(8770, 9)


,UserName,Handle,Timestamp,Content,Likes,Retweets,Emojis,Comments,Tag
0,Michael Lewis & Friends,@Michael65413248,"""2020-07-31 11:45""",Bloggers commented on #COVID19SG #COVID19Singa...,79.0,41.0,0,0.0,GE2020SG


In [5]:
df.Tag.value_counts().keys()

Index(['general_election_sg', 'singaporevotes', 'SingaporeVotes',
       'general_election_sg2020', 'sgelections', 'GE2020', 'SGElection',
       'Ge2020', 'GE2020SG', 'Singapore General Election', 'wpsg', 'GESG2020',
       'singapore_general_election_2020', 'Singapore GE 2020', 'SGGE2020',
       'PAPSingapore'],
      dtype='object')

from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.corpus import stopwords
print(stopwords.words('english'))
mystopwords=stopwords.words("English") +  + ['general_election_sg', 'SGElection', 'SingaporeVotes', 'sgelections',
       'general_election_sg2020', 'GE2020SG', 'wpsg',
       'singapore_general_election_2020', 'SGGE2020', 'PAPSingapore']
       

In [10]:
import spacy,re
# use spacy english stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from  string  import  punctuation

# use GloVe by default or use googlenews
nlp = spacy.load('en_core_web_sm', vectors='en_google')
#mystopwords_spacy = ##list(STOP_WORDS) +
mystopwords_spacy = ['#general_election_sg', '#SGElection', '#SingaporeVotes', '#sgelections',
       '#general_election_sg2020', '#GE2020SG', '#wpsg','#sgvotes','#ge2020sg '
       '#singapore_general_election_2020', '#SGGE2020', '#PAPSingapore','#GE2020', '#GE2020SG']

#define func

# tokenize the text, with the option to remove stopwords and to stem words.
def preprocess(text, remove_stopwords=True, stem_words=False, punctuations=punctuation):
    # use spacy to do nlp process without parser and NER
    doc = nlp(text, disable=['parser', 'ner'])
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
    tokens = [tok for tok in tokens if tok not in mystopwords_spacy and tok not in punctuations]
    text = ' '.join(tokens)
    # Clean the text
    #text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)
    text = re.sub(r'''(?i)\b((?:http?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    # Return a list of words
    
    return(text)
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))


In [11]:
#tokenized_news = [text_tokenizer(text) for text in raw_news]
cleantext = df['Content'].apply(preprocess)

In [12]:
idx = 0
print(df['Content'][idx])
print("*"*20)
print(cleantext[idx])

Bloggers commented on #COVID19SG #COVID19Singapore & #Singapore #GE2020SG #GE2020 went missing before/after 10 July 2020 http://emmanuel-maria.biz/index_files/Page2494.htm…
The #Singaporeans are actively searching for them! Please pray for their safety & we hope to find them quickly. Please RT. Many Thanks.
********************
blogger comment on covid19sg covid19singapore singapore ge2020sg ge2020 go missing before after 10 july 2020     the singaporeans be actively search for please pray for safety hope to find quickly please rt many thank


In [13]:
idx = 10
print(df['Content'][idx])
print("*"*20)
print(cleantext[idx])

'#PAP has to better understand millennials to win back #votes, says Ambassador-at-Large Chan Heng Chee' #COVID19 #ge2020sg #singapore #politics #Opinion
********************
pap have to well understand millennial to win back vote say ambassador at large chan heng chee covid19 ge2020sg singapore politic opinion


In [14]:
cleantext.apply(sentiment_analyzer_scores)

blogger comment on covid19sg covid19singapore singapore ge2020sg ge2020 go missing before after 10 july 2020     the singaporeans be actively search for please pray for safety hope to find quickly please rt many thank {'neg': 0.049, 'neu': 0.561, 'pos': 0.39, 'compound': 0.9217}
feel that can just keep the narrative and deny everything else this ge2020sg show that msm have fail the entire duration of the husting never spend a single second on msm yes be laser focusse on campaigning but everyone know ignore msm {'neg': 0.153, 'neu': 0.808, 'pos': 0.039, 'compound': -0.6542}
the legend of singapore opposition movement ge2025 ge2020sg {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
regardless of race language or religion the topic of race and language take some unforeseen spotlight during ge2020 sutd senior lecturer dr nazry bahrawi join other expert to weigh in on this commonly skirt subject   sutdhumanitie ge2020sg {'neg': 0.0, 'neu': 0.942, 'pos': 0.058, 'compound': 0.296}
senior

the good performance from the pap be mr tharman with 74 62 say a lot do not ge2020sg {'neg': 0.0, 'neu': 0.854, 'pos': 0.146, 'compound': 0.4404}
be not from sengkang but thank you sengkang resident jamus deserve the vote   ge2020sg {'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound': 0.5023}
why be ivan lim in jurong grc  s victory speech who vote for ge2020sg ge2020 sgelection {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
he   so cute pls you win warms the cockles of our heart ahhahaha ge2020sg {'neg': 0.0, 'neu': 0.375, 'pos': 0.625, 'compound': 0.9322}
marymount all finish area clean and 5bx liao or not sgelection ge2020 ge2020sg {'neg': 0.0, 'neu': 0.816, 'pos': 0.184, 'compound': 0.4019}
ge2020sg opinion aside let see if sg can win china in this friendly china have be run riot vs europe so   a tough task {'neg': 0.163, 'neu': 0.607, 'pos': 0.229, 'compound': 0.4891}
i blame the pioneer generation for this fuckery ge2020sg sgelection {'neg': 0.211, 'neu': 0.789, 'pos': 0.0, 'c

sample count be stress out -  should feel hopeful or resign ge2020sg {'neg': 0.297, 'neu': 0.514, 'pos': 0.189, 'compound': -0.2263}
apparently the sound of sengkang resident when the sample count come out for sengkang grc wp 53 pap 47 sgelection ge2020 ge2020sg {'neg': 0.0, 'neu': 0.913, 'pos': 0.087, 'compound': 0.25}
 would  be proud too cause get dr  jamuslim ge2020 ge2020sg singaporeelection {'neg': 0.0, 'neu': 0.763, 'pos': 0.237, 'compound': 0.4767}
ge2020 ge2020sg singaporeelection------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
why would people not vote for dr paul tambyah be a doctor and professor of infectious disease  would  make a great mp ge2020 ge2020sg singaporeelection singaporevotes {'neg': 0.0, 'neu': 0.859, 'pos': 0.141, 'compound': 0.6249}
happy ge2020 sorry i could not include every party xoxo singaporevotes ge2020sg singaporeelection {'neg': 0.176, 'neu': 0.444, 'pos': 0.38, 'compound': 0.7304}
brb cry ge2020 ge2020sg singaporeelection singaporevotes

singapore thank to can now foresee how malaysia come general election will turn out to be {'neg': 0.0, 'neu': 0.857, 'pos': 0.143, 'compound': 0.3612}
dtn singapore pap fighting for transformation new unity this general election be about the transformation o       {'neg': 0.143, 'neu': 0.857, 'pos': 0.0, 'compound': -0.3612}
singapore pap fighting for transformation new unity this general election be about the transformation of t       {'neg': 0.143, 'neu': 0.857, 'pos': 0.0, 'compound': -0.3612}
singapore general election what to watch for tomorrow       {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
result of singapore general election 2020 tomorrow 2 4 million singaporeans will get to vote voting be secre       {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
singapore general election what to watch for tomorrow   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
singapore polling day result of singapore general election 2020 singapore short stories have predict with

singapore hold general election voter in singapore go to the poll today in the most hotly contest genera       {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
rt  bbcnews singapore hold general election   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
rt  bbcnews singapore hold general election   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
singapore hold general election  ------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
singapore hold general election source article from bbc news world 6 may 2020 last update at 19 : 06 et       {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
 asiatodaynews poll open in singapore general election polling station have open for sin       singapore news {'neg': 0.205, 'neu': 0.795, 'pos': 0.0, 'compound': -0.5574}
singapore hold general election voter in singapore go to the poll today in the most hotly contest general e       {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
finally    finally    singapore gene

...

another well time conviction after the ge2020 good job to the establishment     ge2020 singapore covid senior constituency manager who cheat rc of nearly 6 000 in fund jail 12 week     via  yahoosenior constituency manager who cheat rc of nearly 6 000 in fund jail 12 week a senior constituency manager of the residents   committee under pasir ris punggol town council be jail after cheat an rc into disburse nearly 6 000 in public fund sg news yahoo com {'neg': 0.103, 'neu': 0.839, 'pos': 0.057, 'compound': -0.6124}
singapore hold 2020 general elections and the opposition party gain historic ground for the first time   asean singapore trade ge2020 singaporeelectionsingapore   2020 general elections have the status quo change on july 10 2020 singapore hold general election amid the covid - 19 pandemic and opposition party gain historic ground for the first time aseanbriefing com {'neg': 0.0, 'neu': 0.807, 'pos': 0.193, 'compound': 0.9042}
after singapore ge2020 a time for soul search   sin

let please never ever take elect opposition for grant tonight victory have help sleep easy tomorrow need to make sure have back ge2020 singaporevotes sgelection sgpolitic singapore sgge 2020 what the electorate say the online citizen by remy choo zheng xi this election be suppose to be a cakewalk for the pap call in the midst of a pandemic after a 93 billion set of covid - 19 budget the ground could not have be sweet      onlinecitizenasia com {'neg': 0.028, 'neu': 0.826, 'pos': 0.146, 'compound': 0.8489}
singapore   new heartland ge2020-------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
canada extend good wish to the pm and administration and look forward to continue to work with the government and people of singapore to deepen the long stand friendship and collaboration between country canada singapore sg ge2020 {'neg': 0.0, 'neu': 0.764, 'pos': 0.236, 'compound': 0.8481}
canada congratulate singapore  s people  s action party and prime minister lee hsien loong on the outc

singapore ge2020 papsingapore win marine parade grc with 57 76 of vote wpsg have 42 24 singaporevotessingapore ge2020 live result get real time update on singapore  s ge2020 result and full analysis of how the political party fare in the general election   stvisual via  stcom straitstimes com {'neg': 0.0, 'neu': 0.874, 'pos': 0.126, 'compound': 0.7579}
singapore parliament election final result west coast grc pap conservative 51 69  - 26 8 psp centrist 48 31 new pap hold   -  vs 2015 election singapore singaporevotes ge2020 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
as ge2020 result release be over just wait for result of singapore covid19 local transmission number over the weekend and if go back to phase 1 of circuitbreakersg {'neg': 0.0, 'neu': 0.954, 'pos': 0.046, 'compound': 0.0772}
singapore ge2020 papsingapore win sembawang grc with 67 29 of vote nsp sg have 32 71 singaporevotessingapore ge2020 live result get real time update on singapore  s ge2020 result and full ana

go digital this ge2020 definitely benefit capable opposition candidate view power of social singapore {'neg': 0.0, 'neu': 0.57, 'pos': 0.43, 'compound': 0.8074}
pap  s tin pei ling retain macpherson smc seat with 71 74 of the vote be up against people  s power party  s goh meng seng who get 28 26 singapore ge2020 {'neg': 0.0, 'neu': 0.92, 'pos': 0.08, 'compound': 0.4019}
singapore ge2020 papsingapore win punggol west smc with 60 97 of vote wpsg have 39 03 singaporevotessingapore ge2020 live result get real time update on singapore  s ge2020 result and full analysis of how the political party fare in the general election   stvisual via  stcom straitstimes com {'neg': 0.0, 'neu': 0.874, 'pos': 0.126, 'compound': 0.7579}
singapore ge2020 papsingapore win radin mas smc with 74 03 of vote thereformparty have 25 97 singaporevotessingapore ge2020 live result get real time update on singapore  s ge2020 result and full analysis of how the political party fare in the general election   stvisual 

0       None
1       None
2       None
3       None
4       None
        ... 
8765    None
8766    None
8767    None
8768    None
8769    None
Name: Content, Length: 8770, dtype: object

In [15]:
idx = 100
from textblob import TextBlob
blob = TextBlob(cleantext[idx])
blob.tags       
for sentence in blob.sentences:
    print(sentence.sentiment.polarity)
    
sentiment_analyzer_scores(cleantext[idx])

0.5
despite humorous intro people should focus on the message have after   quite inspiring ge2020 ge2020sg {'neg': 0.119, 'neu': 0.711, 'pos': 0.169, 'compound': 0.2285}


In [96]:
0.903+0.097

1.0